In [194]:
import pandas as pd
import ast
import re
import numpy as np

In [4]:
txt_train = pd.read_csv('/Users/collin/Desktop/experiment_img_txt-master/text_prcoess/NN_clf/train_pred.csv',index_col = 0)
txt_val = pd.read_csv('/Users/collin/Desktop/experiment_img_txt-master/text_prcoess/NN_clf/val_pred.csv',index_col = 0)
txt_test  = pd.read_csv('/Users/collin/Desktop/experiment_img_txt-master/text_prcoess/NN_clf/test_pred.csv',index_col = 0)
txt_train.head()

,text_name,category_id,pred_label
0,hot_dog_786,55,[55 47 61 41 46]
1,hot_dog_581,55,[55 45 12 13 94]
2,hot_dog_560,55,[55 78 3 4 28]
3,hot_dog_859,55,[55 16 3 49 4]
4,hot_dog_424,55,[55 57 16 48 78]


In [8]:
img_train = pd.read_csv('/Users/collin/Desktop/experiment_img_txt-master/image_process/result/img_train_pred.csv',index_col = 0)
img_test = pd.read_csv('/Users/collin/Desktop/experiment_img_txt-master/image_process/result/img_test_pred.csv',index_col = 0)
img_val = pd.read_csv('/Users/collin/Desktop/experiment_img_txt-master/image_process/result/img_val_pred.csv',index_col = 0)
img_train.head()

,img,pred
0,cheese_plate_512,[16 13 57 9 19]
1,cheese_plate_709,[16 19 46 48 99]
2,cheese_plate_542,[16 6 46 32 47]
3,cheese_plate_97,[16 17 57 51 19]
4,cheese_plate_175,[16 82 46 57 50]


In [154]:
train = img_train.merge(txt_train, left_on = 'img', right_on = 'text_name')
val = img_val.merge(txt_val, left_on = 'img', right_on = 'text_name')
test = img_test.merge(txt_test, left_on = 'img', right_on = 'text_name')
train= train.dropna()
len(train)

60185

In [336]:
def ensemble(pred, pred_label):
    size = len(pred)
    result = []
    for i in range(size):
        d1 = {}
        d2 = {}
        for j in range(5):
            d1[pred[i][j]] = 5 - j 
        for j in range(5):
            d2[pred_label[i][j]] = 5 - j + 1/(j+1)
        for key in d2.keys():
            d1[key] = d1.get(key,0) + d2[key]
        sorted_x = sorted(d1.items(), key=operator.itemgetter(1))
        result.append(sorted_x[-1][0])
    return np.array(result)

In [319]:
def translate_list(str_list):
    str_list = re.sub(r'\[\s+', '[', str_list)
    return re.sub('\s+', ',', str_list)

In [320]:
train['pred'] = train['pred'].apply(translate_list)
train['pred_label'] = train['pred_label'].apply(translate_list)
train.head(1)

,img,pred,text_name,category_id,pred_label
0,cheese_plate_512,"[16,13,57,9,19]",cheese_plate_512,16,"[16,28,3,10,4]"


In [321]:
train_pred = [ast.literal_eval(item) for item in train['pred'].values]
train_pred_label = [ast.literal_eval(item) for item in train['pred_label'].values]

In [322]:
result = ensemble(train_pred, train_pred_label)
label = train['category_id'].values
print(result.shape)
np.mean(label == result)

(60185,)


0.96286450112154187

In [323]:
label = train['category_id'].values
np.mean(label == result)

0.96286450112154187

In [324]:
val['pred'] = val['pred'].apply(translate_list)
val['pred_label'] = val['pred_label'].apply(translate_list)
val_pred = [ast.literal_eval(item) for item in val['pred'].values]
val_pred_label = [ast.literal_eval(item) for item in val['pred_label'].values]

In [338]:
result = ensemble(val_pred, val_pred_label)
print(result)
label = val['category_id'].values
np.mean(label == result)

[62  8 16 ..., 55 69 55]


0.80540835411471323

In [337]:
test['pred'] = test['pred'].apply(translate_list)
test['pred_label'] = test['pred_label'].apply(translate_list)
pred = [ast.literal_eval(item) for item in test['pred'].values]
pred_label = [ast.literal_eval(item) for item in test['pred_label'].values]
result = ensemble(pred, pred_label)
print(result)
label = test['category_id'].values
np.mean(label == result)

[36 16 16 ..., 55 55 55]


0.80186335403726705